In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16413487
paper_name = 'alto_dixon_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits.txt', header=None, names=['genes'])

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 3 x 1


In [7]:
original_data.head()

,genes
0,BCK1
1,SLT2
2,RLM1


In [8]:
original_data['genes'] = original_data['genes'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [genes, orf]
Index: []


In [12]:
original_data['data'] = 1

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(3, 1)

# Load & process tested strains

In [19]:
tested = pd.read_csv('raw_data/FG_array_genes.txt', sep='\t', header=None)

In [20]:
tested.head()

,0
0,YAL002W
1,YAL004W
2,YAL005C
3,YAL007C
4,YAL008W


In [21]:
tested['orf'] = tested[0].astype(str)

In [22]:
tested['orf'] = clean_orf(tested['orf'])

In [23]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [24]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [0, orf]
Index: []


In [25]:
tested_orfs = tested['orf'].unique()

In [26]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [27]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [28]:
data = original_data.copy()

In [29]:
dataset_ids = [561]
datasets = datasets.reindex(index=dataset_ids)

In [30]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [31]:
data.head()

dataset_id,561
data_type,value
orf,
YAL002W,0
YAL004W,0
YAL005C,0
YAL007C,0
YAL008W,0


## Subset to the genes currently in SGD

In [32]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [33]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,561
,data_type,value
gene_id,orf,
2,YAL002W,0
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [34]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [35]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [36]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        561       
data_type       value valuez
gene_id orf                 
2       YAL002W     0    0.0
1863    YAL004W     0    0.0
4       YAL005C     0    0.0
5       YAL007C     0    0.0
6       YAL008W     0    0.0

# Print out

In [37]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [38]:
from IO.save_data_to_db3 import *

In [39]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 16413487...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.02s/it]

Updating the data_modified_on field...
